In [1]:
import numpy as np
import pandas as pd

In [2]:
train_postulantes_educacion = pd.read_csv("../fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv")
train_postulantes_genero_y_edad = pd.read_csv("../fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv", parse_dates=['fechanacimiento'], date_parser=lambda d: pd.to_datetime(d, format="%Y/%m/%d", errors="coerce"))
train_visitas = pd.read_csv("../fiuba_hasta_15_abril/fiuba_3_vistas.csv")
train_postulaciones = pd.read_csv("../fiuba_hasta_15_abril/fiuba_4_postulaciones.csv")
train_avisos_detalle = pd.read_csv("../fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv")

In [3]:
test_postulantes_educacion = pd.read_csv("../fiuba_desde_15_abril/fiuba_1_postulantes_educacion.csv")
test_postulantes_genero_y_edad = pd.read_csv("../fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv", parse_dates=['fechanacimiento'], date_parser=lambda d: pd.to_datetime(d, format="%Y/%m/%d", errors="coerce"))
test_visitas = pd.read_csv("../fiuba_desde_15_abril/fiuba_3_vistas.csv")
test_postulaciones = pd.read_csv("../test_final_100k.csv")
test_avisos_detalle = pd.read_csv("../fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv")

In [4]:
# Concat train and test. If there are any duplicates, keep the test register.
train_postulantes_educacion = pd.concat([test_postulantes_educacion, train_postulantes_educacion]).drop_duplicates('idpostulante')
train_postulantes_genero_y_edad = pd.concat([test_postulantes_genero_y_edad, train_postulantes_genero_y_edad]).drop_duplicates('idpostulante')
#train_postulaciones = test_postulaciones
train_avisos_detalle = pd.concat([test_avisos_detalle, train_avisos_detalle]).drop_duplicates('idaviso')

# Preparacion de datos

## Postulantes

In [5]:
def process_postulantes(df_educacion, df_genero_edad):
    # Create new column educacion.
    df_educacion['educacion'] = df_educacion['nombre'] + df_educacion['estado']

    # Replace strings by ordinal values.
    df_educacion['educacion'].replace({
        'SecundarioAbandonado': 0,
        'SecundarioEn Curso': 1,
        'SecundarioGraduado': 2,
        'OtroAbandonado': 3,
        'OtroEn Curso': 4,
        'OtroGraduado': 5,
        'Terciario/TécnicoAbandonado': 6,
        'Terciario/TécnicoEn Curso': 7,
        'Terciario/TécnicoGraduado': 8,
        'UniversitarioAbandonado': 9,
        'UniversitarioEn Curso': 10,
        'UniversitarioGraduado': 11,
        'PosgradoAbandonado': 12,
        'PosgradoEn Curso': 13,
        'PosgradoGraduado': 14,
        'MasterAbandonado': 15,
        'MasterEn Curso': 16,
        'MasterGraduado': 17,
        'DoctoradoAbandonado': 18,
        'DoctoradoEn Curso': 19,
        'DoctoradoGraduado': 20,
    }, inplace=True)

    # Sort by the new ordinal value.
    df_educacion.sort_values('educacion', ascending=False, inplace=True)

    # Keep the first occurency on each postulante, which is the greatest education level achieved. 
    df_educacion.drop_duplicates('idpostulante', inplace=True)

    # Check is there are any duplicate in the df_genero_edad DF.
    df_genero_edad.drop_duplicates('idpostulante', inplace=True)

    # Add edad column.
    df_genero_edad['edad'] = np.floor((pd.datetime.today() - df_genero_edad['fechanacimiento']).dt.days / 365)

    # Convert sexo to boolean.
    df_genero_edad['sexo'] = df_genero_edad['sexo'].replace({'FEM': 0, 'MASC': 1}).astype(bool)

    # Remove fechanacimiento.
    del df_genero_edad['fechanacimiento']

    # Merge and return both DF.
    postulantes = pd.merge(df_genero_edad, df_educacion, how='left', on='idpostulante')

    postulantes.columns = ['idpostulante', 'sexo', 'edad', 'educacion_nivel', 'educacion_estado', 'educacion']

    return postulantes

### Train dataframe

In [6]:
train_postulantes = process_postulantes(train_postulantes_educacion, train_postulantes_genero_y_edad)

In [7]:
# Size.
len(train_postulantes)

420490

In [8]:
# Check for rows with null values.
train_postulantes.isnull().sum()

idpostulante            0
sexo                    0
edad                23546
educacion_nivel     49766
educacion_estado    49766
educacion           49766
dtype: int64

In [9]:
postulantes = train_postulantes

In [10]:
postulantes.sample(5)

,idpostulante,sexo,edad,educacion_nivel,educacion_estado,educacion
219110,mzdZRdY,True,NaN,NaN,NaN,NaN
153782,ow5mrjj,True,25.0,Secundario,Graduado,2.0
122255,LNKVO9b,False,24.0,Universitario,En Curso,10.0
144965,QNr8WRq,True,56.0,NaN,NaN,NaN
52745,X95YRYJ,True,29.0,Universitario,Graduado,11.0


In [11]:
postulantes.to_pickle("postulantes.pkl")

## Avisos

In [12]:
# Check for duplicates
print(len(train_avisos_detalle))
print(len(train_avisos_detalle.drop_duplicates('idaviso')))

24181
24181


In [13]:
avisos = train_avisos_detalle

In [14]:
for column in ['idpais', 'ciudad', 'mapacalle']:
    del avisos[column]

In [15]:
avisos.sample(5)

,idaviso,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
11814,1112320063,11 Feb - Online Hiring Tournament - QA Enginee...,<p>Para inscribirse en el torneo debe ingresar...,Gran Buenos Aires,Teletrabajo,Senior / Semi-Senior,Tecnologia / Sistemas,CrossOver
9680,1112494542,Asesor Comercial- Full Time,<p>¿Te gustaría trabajar en una empresa certif...,Gran Buenos Aires,Full-time,Junior,Ventas,RESUELVE
6889,1112432524,SOLDADOR / ARMADOR DE CARROCERÍAS (ROSARIO - S...,<p>Estamos en la busqueda de dos profesionales...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Producción,Grupo Innovar
911,1112345552,Abogado/a recién recibido zona berazategui,IMPORTANTE ESTUDIO DE BERAZATEGUI SE ENCUENTRA...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Legal,Estudio Juridico Escobar y Asociados
8724,1112094914,Oficial de Cuentas Corporativas / Ejecutivo Co...,<p>Importante Compañía Financiera ubicada en M...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,CREDIBEL


In [16]:
avisos.to_pickle("avisos.pkl")

## Visitas

No podemos concatenar los datasets de visitas, habria conflicto en los casos en los que en train no hubo postulacion, pero en test (mas adelante) si hubo visita, estariamos asignando una visita futura a una no postulacion en el train.

In [17]:
def visitas_process(visitas):
    visitas2 = visitas.groupby(['idAviso', 'idpostulante']).agg({
        'timestamp': len,
        'idAviso': lambda x: x.iloc[0],
        'idpostulante': lambda x: x.iloc[0]
    })
    
    visitas2.reset_index(drop=True, inplace=True)
    visitas2.columns = ['visita_cantidad', 'idaviso', 'idpostulante']
    
    return visitas2

In [18]:
%%time
train_visitas2 = visitas_process(train_visitas)

CPU times: user 3min 30s, sys: 1.51 s, total: 3min 32s
Wall time: 3min 32s


In [19]:
%%time
test_visitas2 = visitas_process(test_visitas)

CPU times: user 6min 17s, sys: 3.7 s, total: 6min 20s
Wall time: 6min 21s


In [20]:
len(train_visitas2)

2983869

In [ ]:
len(test_visitas2)

In [21]:
train_visitas2.head(5)

,visita_cantidad,idaviso,idpostulante
0,2,12543760,1Q4KX9b
1,1,12543760,5lYQqw
2,2,12543760,8M26bzW
3,4,12543760,DrJ10XL
4,1,12543760,LNKkl1b


In [22]:
train_visitas2.to_pickle('visitas_train.pkl')
test_visitas2.to_pickle('visitas_test.pkl')

## Postulaciones

In [23]:
postulaciones = train_postulaciones.drop_duplicates(['idaviso', 'idpostulante'])

In [25]:
del postulaciones['fechapostulacion']
postulaciones.loc[:, "target"] = True

/home/jose/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/jose/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [26]:
len(postulaciones)

4909025

In [27]:
postulaciones.to_pickle("postulaciones_train.pkl")

Para estos datasets, existen postulaciones que no tienen visitas, esto no deberia ser posible. Asumimos que el dataset train de visitas esta incompleto, y suponemos que el dataset test de visitas estara en mejores condiciones. Por lo tanto hacemos un **merge** de tipo **inner**, el cual deja afuera a las postulaciones sin visitas.

Otra forma seria asignar una visita a las postulaciones que no tienen. El usuario tiene que generar una visita antes de poder postularse, por lo que tiene sentido.

In [28]:
postulaciones = pd.merge(postulaciones, train_visitas2, how='inner', on=['idaviso', 'idpostulante'])

In [29]:
postulaciones.head(5)

,idaviso,idpostulante,target,visita_cantidad
0,1112383420,ZaO5,True,2
1,1112397910,ZaO5,True,2
2,1111043912,ZaO5,True,3
3,1112421913,ZaO5,True,2
4,1112373820,ZaO5,True,1


### Casos negativos

Generamos al azar combinaciones de **idaviso** e **idpostulante**, la misma cantidad que el set de casos positivos. Luego descartamos las combinaciones que existen en el DF de casos positivos (de existir seran muy pocas, ya que la probabilidad de un caso positivo en el conjunto de todas las combinaciones posibles es muy bajo).

Las combinaciones resultantes son el conjunto de postulaciones que nunca se haran, mas las postulaciones que si se haran en el futuro. Este modelo supone que de existir postulaciones futuras en las combinaciones generadas, estas son pocas, por lo tanto, el conjunto de features de avisos y postulantes representa en su mayoria a un set de variables independientes con poca tendencia a generar una postulacion.

Este supuesto se pondra a prueba (a grosso modo) al ejecutar los algoritmos predictivos. De ser valido obtendremos un score operativo.

In [31]:
len(avisos['idaviso'].unique())

24181

In [32]:
len(postulantes['idpostulante'].unique())

420490

In [33]:
# Create a hash table for fast lookup of existing records.
idaviso_idpostulante = {i for i in (avisos['idaviso'].astype(str) +  postulantes['idpostulante']).values}

In [34]:
avisos_values = avisos['idaviso'].unique()
postulantes_values = postulantes['idpostulante'].unique()

n = len(postulaciones)

avisos_index = np.random.choice(len(avisos_values), n)
postulantes_index = np.random.choice(len(postulantes_values), n)

In [35]:
# See https://stackoverflow.com/questions/10715965/add-one-row-in-a-pandas-dataframe#answer-17496530

postulaciones_2 = []

for i in range(n):
    if (str(avisos_values[avisos_index[1]]) + postulantes_values[postulantes_index[i]] not in idaviso_idpostulante):
        postulaciones_2.append({
            'idaviso': avisos_values[avisos_index[i]],
            'idpostulante': postulantes_values[postulantes_index[i]]
        })

postulaciones_2 = pd.DataFrame(postulaciones_2, columns=['idaviso', 'idpostulante'])

In [36]:
postulaciones_2['target'] = False

Aqui asignamos 0 a las visitas que no tienen valor, estamos suponiendo que para los casos generados, la mayoria son no postulaciones, y que la mayoria de las no postulaciones, no tienen visitas.

In [37]:
postulaciones_2 = pd.merge(postulaciones_2, train_visitas2, how='left', on=['idaviso', 'idpostulante'])
postulaciones_2.loc[postulaciones_2['visita_cantidad'].isnull(), 'visita_cantidad'] = 0

In [38]:
len(postulaciones_2)

1398718

In [39]:
# Merge both DFs and shuffle the false and true cases.
postulaciones = pd.concat([postulaciones, postulaciones_2]).sample(frac=1)
postulaciones.reset_index(inplace=True, drop=True)

In [40]:
postulaciones.sample(5)

,idaviso,idpostulante,target,visita_cantidad
1299300,1112389785,mzdBQVE,False,0.0
1169688,1112389495,1pO5wL,True,4.0
1551304,1112481338,96X8xWv,False,0.0
1564275,1112048953,NzjaJQB,False,0.0
2667920,1111926762,JBraMx5,True,2.0


In [41]:
postulaciones.to_pickle("postulaciones_visitas_train.pkl")